In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset= pd.read_csv("mushrooms.csv")
dataset = dataset[dataset['stalk-root'] != '?']

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
ds=dataset.apply(LabelEncoder().fit_transform)

In [4]:
df=pd.DataFrame(ds[['classes', 'cap-shape', 'cap-surface','cap-color', 'bruises', 'odor','gill-attachment', 'gill-spacing', 'gill-size', 'gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring', 'stalk-color-above-ring','stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number','ring-type','spore-print-color', 'population', 'habitat']])
df.head()

,classes,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,2,...,2,5,5,0,0,1,3,1,3,5
1,0,5,2,7,1,0,1,0,0,2,...,2,5,5,0,0,1,3,2,2,1
2,0,0,2,6,1,3,1,0,0,3,...,2,5,5,0,0,1,3,2,2,3
3,1,5,3,6,1,6,1,0,1,3,...,2,5,5,0,0,1,3,1,3,5
4,0,5,2,3,0,5,1,1,0,2,...,2,5,5,0,0,1,0,2,0,1


In [5]:
from sklearn.preprocessing import StandardScaler
x = pd.DataFrame(df.iloc[:, 1:23].values)
y = pd.DataFrame(df.iloc[:, 0:1].values)
x_enc=x
scaler = StandardScaler()
x_std = scaler.fit_transform(x_enc)
y_enc = le.fit_transform(y.values.ravel())
print (x_enc.head())
print (y.head())

   0   1   2   3   4   5   6   7   8   9  ...  12  13  14  15  16  17  18  19  \
0   5   2   4   1   6   1   0   1   2   0 ...   2   5   5   0   0   1   3   1   
1   5   2   7   1   0   1   0   0   2   0 ...   2   5   5   0   0   1   3   2   
2   0   2   6   1   3   1   0   0   3   0 ...   2   5   5   0   0   1   3   2   
3   5   3   6   1   6   1   0   1   3   0 ...   2   5   5   0   0   1   3   1   
4   5   2   3   0   5   1   1   0   2   1 ...   2   5   5   0   0   1   0   2   

   20  21  
0   3   5  
1   2   1  
2   2   3  
3   3   5  
4   0   1  

[5 rows x 22 columns]
   0
0  1
1  0
2  0
3  1
4  0


In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_std, y_enc, test_size = 0.25, random_state = 0)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
clf = LogisticRegression()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
metrics.f1_score(y_test, y_pred)

0.9772727272727273

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
param_grid = [{'C': np.logspace(-3, 3, 20)}]

grid_search = GridSearchCV(estimator=LogisticRegression(),param_grid=param_grid,cv=StratifiedShuffleSplit(n_splits=20, test_size=0.2, random_state=42),
    scoring='f1',
    n_jobs=-1)

scores = cross_val_score(estimator=grid_search,X=x_std,y=y_enc,cv=StratifiedShuffleSplit(n_splits=20, test_size=0.2, random_state=0),
    scoring='f1',
    n_jobs=-1)

In [9]:
scores

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])

In [10]:
grid_search.fit(x_std, y_enc)
grid_search.best_params_

{'C': 26.366508987303554}

In [11]:
final_clf = LogisticRegression(C=26.36)
final_clf.fit(x_std, y_enc);

In [12]:
feature_ranks = pd.DataFrame(final_clf.coef_, index=['parameter value'])
feature_ranks.columns = x_enc.columns
feature_ranks.sort_values('parameter value', axis=1, ascending=False).T.head()

,parameter value
7,18.155782
18,13.596889
3,4.013930
16,3.605560
1,0.686893


In [13]:
feature_ranks.sort_values('parameter value', axis=1, ascending=True).T.head()

,parameter value
10,-33.318155
6,-26.152246
9,-13.818172
11,-12.456171
20,-3.028005
